# CMSE 381 Final Project Template

**INSTRUCTIONS**: This is a template to help organize your project.  All projects should include the 5 major sections below (you do not need to use this template file).  If you use this file, complete your work below and remove content in parentheses. Also, remove this current cell.  

#### CMSE 381 Final Project
### &#9989; Group members: Colby Degan, Jose Villegas
### &#9989; Section_002
#### &#9989; 4/25/2025

# Concrete Crack Detection using SVCs and CNNs

## Background and Motivation

_(Provide context for the problem.  **Clearly state the question(s) you set
out to answer.**)_

We wanted to create a model that could accurately notice small differences in images and correctly classify them based on that decision. To do this we decided to use images of concrete, some of which have cracks, to make a model that could achieve this goal.

## Methodology
_(How did you go about answering your question(s)? You should wrote some code here to demonstrate what the data is like and how in principle your method works. You can leave the variations of the related to specific results to the results section.)_

In [1]:
# you may want to import some modules here
import matplotlib as mpl
import matplotlib.image as img
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import operator
import os
import pandas as pd
import PIL as pillow
import random
import seaborn as sns
import sklearn
import sqlite3
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
import time
import urllib.request
import warnings 
warnings.filterwarnings('ignore')

from IPython import display
from IPython.display import clear_output, display, set_matplotlib_formats
from itertools import combinations
from matplotlib.colors import ListedColormap
from PIL import Image
from random import choice
from sklearn import datasets, decomposition, linear_model, metrics, model_selection, neighbors, pipeline, preprocessing, svm, tree, utils
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, LogisticRegression, Perceptron, Ridge, RidgeCV, SGDRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, scale, SplineTransformer, StandardScaler
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.utils import shuffle

### Data
_(Describe the data you are using. What variables are you using? What they mean? Why did you choose them?)_

We are compressing each 256x256 image into 64x64, and are using each pixel measurement of that compressed image as a variable, giving us 4096 pixel measurements. We are also using the type of structure as a variable, as it wouldn't be logical to compare a picture of cracked pavement to a picture of an un-cracked wall.

In [ ]:
# you may need some code here to adjust the format or organization of your data so that they can be used for the model

In [ ]:
# you may want to write some code to see what the data looks like

In [2]:
folder_paths = ["archive/Decks/Cracked", "archive/Decks/Non-cracked", "archive/Pavements/Cracked", "archive/Pavements/Non-cracked", "archive/Walls/Cracked", "archive/Walls/Non-cracked"]
image_data = []
cracked = []
structure = []

# For every folder that contains images
for folder_path in folder_paths:
    
    # Get list of image files
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

    # Initialize list to store image data
    for i in range(len(image_files)):
        img_path = os.path.join(folder_path, image_files[i])
        img = Image.open(img_path).convert('L')
        img.thumbnail((64, 64), Image.Resampling.LANCZOS)
        img_array = np.array(img).flatten()
        image_data.append(img_array)
        if folder_path == "archive/Decks/Cracked" or folder_path == "archive/Decks/Non-cracked":
            structure.append("Deck")
        elif folder_path == "archive/Pavements/Cracked" or folder_path == "archive/Pavements/Non-cracked":
            structure.append("Pavement")
        else:
            structure.append("Wall")
        if folder_path == "archive/Decks/Cracked" or folder_path == "archive/Pavements/Cracked" or folder_path == "archive/Walls/Cracked":
            cracked.append(1)
        else:
            cracked.append(0)
              
                
df = pd.DataFrame(image_data)
df["Structure"] = structure
df["Cracked"] = cracked
df.to_csv("concrete_dataset.csv", index=False)

In [3]:
concrete_dataset = pd.read_csv('concrete_dataset.csv')

In [4]:
concrete_dataset

,0,1,2,3,4,5,6,7,8,9,...,4088,4089,4090,4091,4092,4093,4094,4095,Structure,Cracked
0,185,185,158,182,182,169,160,175,169,175,...,182,182,183,185,186,182,172,180,Deck,1
1,178,182,171,183,181,179,183,173,178,176,...,172,186,197,185,173,179,179,183,Deck,1
2,181,184,190,187,186,190,190,184,186,184,...,191,186,184,181,186,189,181,184,Deck,1
3,191,201,202,192,194,188,191,190,181,171,...,181,179,177,178,174,177,182,179,Deck,1
4,180,172,179,171,185,178,177,183,185,183,...,176,177,179,179,180,183,177,175,Deck,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56087,144,141,146,145,141,147,149,151,150,147,...,144,146,147,147,148,143,144,150,Wall,0
56088,176,177,177,178,178,180,180,180,180,180,...,175,176,176,177,178,175,176,175,Wall,0
56089,159,162,168,159,154,169,169,160,159,155,...,170,172,162,154,151,160,160,146,Wall,0
56090,150,171,182,184,185,182,179,183,183,181,...,187,185,185,187,188,186,184,185,Wall,0


### Models for classification _(if applicable)_
_(What models will you be using for classification? Why did you choose to use them? What questions would you answer with them? How would you evaluate if each model? What cross-validation method did you use?)_

In [ ]:
# you may add some code here to show how the model works in principle

### Other methods used _(if applicable)_

_(If this is a preprocessing step to prepare your data for regression or classification models, you should put this subsection before your explanation for the regression or classification models.)_

_(What method did you use otherwise? Why did you choose to use them? What questions would you answer with them? How would you evaluate the results? What cross-validation method did you use when applicable?)_

In [ ]:
# you may add some code here to show how the method works in principle

# you may add some code here to show how the model works in principle

## Results

_(What did you find when you carried out your methods? Some of your code related to
presenting results/figures/data may be replicated from the methods section or may only be present in
this section. All of the plots that you plan on using for your presentation should be present in this
section)_

### classification results
_(What are you trying to do here?)_

In [ ]:
# how did you do it

_(How do you interpret what you see?)_

_(What are you doing next?)_

In [ ]:
# how did you do it (etc. etc.)

### other results
_(What are you trying to do here?)_

In [ ]:
# how did you do it

_(How do you interpret what you see?)_

_(What are you doing next?)_

In [ ]:
# how did you do it (etc. etc.)

## Discussion and Conclusion

_(What did you learn from your results? What obstacles did you run into? What would you do differently next time? Clearly provide quantitative answers to your question(s)?  At least one of your questions should be answered with numbers.  That is, it is not sufficient to answer "yes" or "no", but rather to say something quantitative such as variable 1 increased roughly 10% for every 1 year increase in variable 2.)_

### discussion on the regression results

### discussion on the other results

### conclusion and future steps

## Author contribution

_(Please describe the contribution of each member of group)._

## References

_(List the source(s) for any data and/or literature cited in your project.  Ideally, this should be formatted using a formal citation format (MLA or APA or other, your choice!).   Multiple free online citation generators are available such as <a href="http://www.easybib.com/style">http://www.easybib.com/style</a>. **Important:** if you use **any** code that you find on the internet for your project you **must** cite it or you risk losing most/all of the points for you project.)_